<a href="https://colab.research.google.com/github/ShuqairABD/METADATA_EXIF_IPTC_XMP/blob/main/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%A2%D0%B5%D0%B3%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F_(Metadata)_EXIF_IPTC_XMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **start !pip / Libraries**

In [ ]:
!pip install xmp
!pip install exif
!pip install python-xmp-toolkit
!pip install pyexiv2
!pip install piexif
!apt-get install -y exiftool
!pip install iptcinfo3

In [2]:
import os
import shutil
from google.colab import files
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from time import sleep
from ipywidgets import interact_manual, Dropdown, Text
from ipywidgets import Text, Dropdown, Button, widgets
from iptcinfo3 import IPTCInfo
import piexif
from zipfile import ZipFile
from IPython.display import display, clear_output
import zipfile

# from IPython.display import display
# from IPython.display import clear_output

# **Модель**

### **для одного фото**

In [9]:
def add_keyword_xmp(image_path, keyword_list):
    try:
        for keyword in keyword_list:
            os.system(f'exiftool -XMP:Subject+="{keyword}" "{image_path}"')
            print(f"Ключевое слово '{keyword}' добавлено к метаданным XMP {image_path}")
    except Exception as e:
        print(f"Ошибка при добавлении ключевого слова к метаданным XMP {image_path}: {str(e)}")

def add_keywords_to_image_exif(image_path, keywords):
    try:
        exif_dict = piexif.load(image_path)
        existing_keywords = exif_dict["Exif"].get(piexif.ExifIFD.UserComment, b"").decode()
        existing_keywords_list = existing_keywords.split(",") if existing_keywords else ['ASCII    ']
        existing_keywords_list.extend(keywords)
        keyword_str = ''.join(existing_keywords_list[0]) + ",".join(existing_keywords_list[1:])
        exif_ifd = {piexif.ExifIFD.UserComment: f'{keyword_str}'.encode()}
        exif_dict = {"0th": {}, "Exif": exif_ifd, "1st": {}, "thumbnail": None, "GPS": {}}
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)
        print(f"Ключевые слова '{keywords}' добавлены к метаданным EXIF.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным EXIF: {str(e)}")

def add_keywords_to_image_iptc(image_path, keywords):
    try:
        info = IPTCInfo(image_path)
        existing_keywords = info['keywords']
        keywords_utf8 = [keyword.decode('utf-8') if isinstance(keyword, bytes) else keyword for keyword in keywords]
        existing_keywords_str = [kw.decode('utf-8') if isinstance(kw, bytes) else kw for kw in existing_keywords]
        for keyword in keywords_utf8:
            if keyword not in existing_keywords_str:
                existing_keywords_str.append(keyword)
        existing_keywords_bytes = [kw.encode('utf-8') if isinstance(kw, str) else kw for kw in existing_keywords_str]
        info['keywords'] = existing_keywords_bytes
        info.save()
        print(f"Ключевые слова '{keywords}' добавлены к метаданным IPTC.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным IPTC: {str(e)}")

def tag_images(filename, keywords, method):
    tagged_images = []
    try:
        if method == 'XMP':
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                add_keyword_xmp(filename, keywords)
                tagged_images.append(filename)
        elif method == 'EXIF':
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                add_keywords_to_image_exif(filename, keywords)
                tagged_images.append(filename)
        elif method == 'IPTC':
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                add_keywords_to_image_iptc(filename, keywords)
                tagged_images.append(filename)
        elif method == 'All':
            for m in ['XMP', 'EXIF', 'IPTC']:
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    if m == 'XMP':
                        add_keyword_xmp(filename, keywords)
                    elif m == 'EXIF':
                        add_keywords_to_image_exif(filename, keywords)
                    elif m == 'IPTC':
                        add_keywords_to_image_iptc(filename, keywords)
                    tagged_images.append(filename)
        else:
            print("Выбран неверный метод.")
    except Exception as e:
        print(f"Ошибка: {e}")

def interactive_tagging():
    file_upload = widgets.FileUpload(accept='image/*', multiple=False)
    method_dropdown = widgets.Dropdown(options=['All', 'XMP', 'EXIF', 'IPTC'], description='Метод:')
    keyword_text = widgets.Text(value='', description='Ключевые слова:')
    button = widgets.Button(description="Добавить ключевые слова")

    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            if file_upload.value:
                filename = list(file_upload.value.keys())[0]
                file_info = file_upload.value[filename]
                content = file_info['content']

                # Сохранить загруженный файл
                with open(filename, 'wb') as f:
                    f.write(content)

                method = method_dropdown.value
                keywords = keyword_text.value.split(',')
                tag_images(filename, keywords, method)
            else:
                print("Пожалуйста, загрузите файл изображения.")

    button.on_click(on_button_clicked)

    display(file_upload)
    display(method_dropdown)
    display(keyword_text)
    display(button)
    display(output)

interactive_tagging()


FileUpload(value={}, accept='image/*', description='Upload')

Dropdown(description='Метод:', options=('All', 'XMP', 'EXIF', 'IPTC'), value='All')

Text(value='', description='Ключевые слова:')

Button(description='Добавить ключевые слова', style=ButtonStyle())

Output()

### **для папки (много фото)**

In [8]:
def add_keyword_xmp(image_path, keyword_list):
    try:
        for keyword in keyword_list:
            os.system(f'exiftool -XMP:Subject+="{keyword}" "{image_path}"')
            print(f"Ключевое слово '{keyword}' добавлено к метаданным XMP {image_path}")
    except Exception as e:
        print(f"Ошибка при добавлении ключевого слова к метаданным XMP {image_path}: {str(e)}")

def add_keywords_to_image_exif(image_path, keywords):
    try:
        exif_dict = piexif.load(image_path)
        existing_keywords = exif_dict["Exif"].get(piexif.ExifIFD.UserComment, b"").decode()
        existing_keywords_list = existing_keywords.split(",") if existing_keywords else ['ASCII    ']
        existing_keywords_list.extend(keywords)
        keyword_str = ''.join(existing_keywords_list[0]) + ",".join(existing_keywords_list[1:])
        exif_ifd = {piexif.ExifIFD.UserComment: f'{keyword_str}'.encode()}
        exif_dict = {"0th": {}, "Exif": exif_ifd, "1st": {}, "thumbnail": None, "GPS": {}}
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, image_path)
        print(f"Ключевые слова '{keywords}' добавлены к метаданным EXIF.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным EXIF: {str(e)}")

def add_keywords_to_image_iptc(image_path, keywords):
    try:
        info = IPTCInfo(image_path)
        existing_keywords = info['keywords']
        keywords_utf8 = [keyword.decode('utf-8') if isinstance(keyword, bytes) else keyword for keyword in keywords]
        existing_keywords_str = [kw.decode('utf-8') if isinstance(kw, bytes) else kw for kw in existing_keywords]
        for keyword in keywords_utf8:
            if keyword not in existing_keywords_str:
                existing_keywords_str.append(keyword)
        existing_keywords_bytes = [kw.encode('utf-8') if isinstance(kw, str) else kw for kw in existing_keywords_str]
        info['keywords'] = existing_keywords_bytes
        info.save()
        print(f"Ключевые слова '{keywords}' добавлены к метаданным IPTC.")
    except Exception as e:
        print(f"Произошла ошибка при добавлении ключевых слов к метаданным IPTC: {str(e)}")

def tag_images_in_folder(folder_path, keywords, method):
    tagged_images = []
    try:
        output_folder = os.path.join(folder_path, 'tagged')
        os.makedirs(output_folder, exist_ok=True)
        for filename in os.listdir(folder_path):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(folder_path, filename)
                new_file_path = os.path.join(output_folder, filename)
                shutil.copy2(file_path, new_file_path)
                if method == 'XMP':
                    add_keyword_xmp(new_file_path, keywords)
                elif method == 'EXIF':
                    add_keywords_to_image_exif(new_file_path, keywords)
                elif method == 'IPTC':
                    add_keywords_to_image_iptc(new_file_path, keywords)
                elif method == 'All':
                    add_keyword_xmp(new_file_path, keywords)
                    add_keywords_to_image_exif(new_file_path, keywords)
                    add_keywords_to_image_iptc(new_file_path, keywords)
                tagged_images.append(new_file_path)

        zip_filename = os.path.join(folder_path, 'tagged_images.zip')
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in tagged_images:
                zipf.write(file, os.path.basename(file))

        print(f"Обработка завершена. Обработано {len(tagged_images)} изображений.")
        print(f"Файлы сохранены в архив: {zip_filename}")
    except Exception as e:
        print(f"Ошибка: {e}")

def interactive_tagging():
    folder_text = widgets.Text(value='', description='Путь к папке:')
    method_dropdown = widgets.Dropdown(options=['All', 'XMP', 'EXIF', 'IPTC'], description='Метод:')
    keyword_text = widgets.Text(value='', description='Ключевые слова:')
    button = widgets.Button(description="Добавить ключевые слова")
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            folder_path = folder_text.value
            method = method_dropdown.value
            keywords = keyword_text.value.split(',')
            if os.path.isdir(folder_path):
                tag_images_in_folder(folder_path, keywords, method)
            else:
                print("Пожалуйста, введите корректный путь к папке.")

    button.on_click(on_button_clicked)

    display(folder_text)
    display(method_dropdown)
    display(keyword_text)
    display(button)
    display(output)

interactive_tagging()


Text(value='', description='Путь к папке:')

Dropdown(description='Метод:', options=('All', 'XMP', 'EXIF', 'IPTC'), value='All')

Text(value='', description='Ключевые слова:')

Button(description='Добавить ключевые слова', style=ButtonStyle())

Output()